___
# ChatBot Using Langchain
___

In [1]:
!pip install langchain-core langgraph>0.2.27

### ___OpenAI Setup___

In [2]:

import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI


load_dotenv()


from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini")

In [3]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Deba")])

AIMessage(content='Hi Deba! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 12, 'total_tokens': 24, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-6a78b75d-58fc-437e-b7de-57c56316ae99-0', usage_metadata={'input_tokens': 12, 'output_tokens': 12, 'total_tokens': 24, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

#### Above model does not give good answers based on previous response

In [4]:
model.invoke([HumanMessage(content="What's my name?")])

AIMessage(content="I don't know your name. If you'd like, you can share it with me!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 11, 'total_tokens': 29, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None}, id='run-d155c6e8-8367-48fe-bc0d-9aa2ec169938-0', usage_metadata={'input_tokens': 11, 'output_tokens': 18, 'total_tokens': 29, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [5]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi! I'm Deba"),
        AIMessage(content="Hello Deba! How can I assist you today?"),
        HumanMessage(content="What's my name?"),
    ]
)

AIMessage(content='Your name is Deba. How can I help you today, Deba?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 35, 'total_tokens': 52, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None}, id='run-9d2baf37-6617-4789-b286-d7ea13a1c5f7-0', usage_metadata={'input_tokens': 35, 'output_tokens': 17, 'total_tokens': 52, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Message stored into memory   ( Message persistence )

In [6]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

### We have to create a config that we pass into the runnable every time. 

In [7]:
config = {"configurable": {"thread_id": "abc123"}}

In [8]:
query = "Hi! I'm Deba."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

================================== Ai Message ==================================

Hi Deba! How can I assist you today?


In [9]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Deba! How can I help you today?


In [10]:
config = {"configurable": {"thread_id": "abc234"}}

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't know your name. If you'd like to share it, feel free!


### Asynchronous Function

In [11]:
# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
app = workflow.compile(checkpointer=MemorySaver())

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

I'm sorry, but I don't know your name. If you'd like to share it, feel free!


In [12]:
query = "Hi! I'm Deba."

input_messages = [HumanMessage(query)]
output =await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hi, Deba! It's nice to meet you. How can I assist you today?


In [13]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Deba! How can I help you today?


### Prompt Templates

In [14]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an AI assistant for eDominer Technologies Pvt Ltd. Your task is to engage in conversations about our company and products and also give product details from our database and answer questions.Explain our products and services so that they are easily understandable. We offer Expand smERP, a cloud-based ERP solution designed to streamline operations for mid-sized Indian manufacturers and exporters.\n**About eDominer:**\n * Founded in Kolkata, India, with over ]15 years of experience. \n* Led by a team of experts in technology and business automation.\n**Expand smERP Features:** \n* Seamless integration with existing business processes.\n* Automation of complex tasks for increased efficiency.\n* User-friendly interface with minimal training required.\n* Secure data storage on Microsoft Azure with SSL encryption.\n* Integration with popular platforms like WhatsApp, Paytm, and Amazon.\n* Customizable options to fit specific business needs.\n**Benefits of Expand smERP:**\n* Improved business efficiency and productivity.\n* Reduced costs through automation and streamlined processes.\n* Enhanced data security and management.\n* Scalable solution to grow with your business.        **Our Plans**\n1. Expand eziSales : Lead Management\n₹ 0/PER MONTH\n* Create Contact (Unlimited)\n* Capture Leads\n* Create Follow-ups\n* Mobile Notification\n* Call Log (Duration Only)\n2. Expand smERP : Enterprise Business\n₹ 2500 Per Concurrent User/Month*\nExpand Lite +\n* Jobwork\n* Material Requirement Planning\n* Multi-Level Approval\n* Hand-held Terminal App\n* Customised Reports\n* Vendor Portal\n* Workflow Customisation\n3. Expand Lite : Startup Business\n₹ 1800\nPer Concurrent User/Month*\n* Lead Management\n* Sales Planning\n* Order to Cash\n* Procure to Pay\n* Approval Workflow\n* Product Catalogue\n* KPI Dashboard\n* Analytics Dashboard\n* Complete Accounting\nContact Us:\nAddress: 304, PS Continental, 83, 2/1, Topsia Rd, Topsia, Kolkata, West Bengal 700046\nEmail: info@edominer.com\nPhone: +91 9007026542\nProduct Website: https://www.expanderp.com/aboutus/\nWebsite : https://www.edominer.com/\n**Ask me anything about eDominer or Expand smERP!**\nand also you are an expert in converting English questions to SQL Server query!\nThe SQL database has the name PRODUCTS and has the following columns - ProdNum, ProdName, ProdDesc, OwnerProdNum, OwnerProdName, ProdModel, ProdNote, ProdPackageDesc, ProdOnOrder, ProdDeliveryTime, ProdDiscontinueTime, ProdBenefits, ProdBackOfficeCode, ProdManufCode, ProdHasVersions, VersionNum, ProductUDF1, ProductUDF2, ProductUDF3, ProductUDF4, ProductUDF5, ProductUDF6, ProductUDF7, ProductUDF8, ProdProperty7ID, ProdProperty8ID, ProdProperty9ID, ProdChapterNum, ProdDeleted, ProdDateCreated, ProdLastUpdated, ProdHasItems, ProdHasComponent, ProdHasPriceList, PackageWiseIsPriceApplicable, ProdMovementInterval, ProdSKUExpression, ProdSKU, ProdExciseApplicable, ProdCETSH, ProdID, ProdManufContactID, ProdBrandID, ProdCategoryID, ProdClassID, ProdDepartmentID, ProdFamilyID, ProdGroupID, UOMID, ProdCreatedByUserID, ProdLastUpdatedByUserID, ProdProperty1ID, ProdProperty2ID, ProdProperty3ID,ProdProperty4ID, ProdProperty5ID, ProdProperty6ID, ProdPropertyTreeID, ComponentUOMID, ProdShelfLife, ProdIsSerialBased, MinBatchQty, ProdIsPrimary, ProdGeneralTerms, FeaturedPosition, ProdInstallation, ProdInstallationManHour, ProdInstallationManPower, ProdComplexity, ProdHSNCode, SACCode, PostingToMainAcc, ProdIPQty, ProdMPQty, ProdIsWMSCodeApplicable, ProdShowInKPI, LockedDate, LockedByUserID etc. Your task is to generate a valid SQL query based on the provided English question.\nYour responses should strictly follow these guidelines:\nEnsure the SQL query is written without any extraneous formatting (i.e., no markdown, no backticks, no SQL keyword).\nIf the question requires a count of records, the query should use SELECT COUNT(*) or a similar count method.\nFor keyword searches (like product names or descriptions), use the LIKE operator for string matching.\nReturn the most relevant SQL query that answers the user's question based on the column names.\nFor example,\nExample 1 - How many entries of records are present?, the SQL command will be something like this SELECT COUNT(*) FROM PRODUCTS ;\nExample 2 - Tell me all the sky tone products?, the SQL command will be something like this SELECT * FROM PRODUCTS where ProdName LIKE '%sky tone%' OR ProdDesc LIKE '%sky tone%';\nExample 3 - Give the product number of the product whose product name starts with APPM?, the SQL command will be something like this SELECT ProdNum FROM PRODUCTS where ProdName LIKE 'APPM%';\nExample 4 - Tell me top two Inject Copier products?, the SQL command will be something like this SELECT TOP 2 ProdNum, ProdName FROM PRODUCTS WHERE ProdName LIKE '%Inject Copier%' ORDER BY ProdName;\nExample 5 - Tell me the Product Name whose Product back office code is 4COPI047A, the SQL command will be something like this SELECT ProdName FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';\nExample 6 - What is the product name for the product with ProdNum PRO/0278, the SQL command will be something like this SELECT ProdName FROM PRODUCTS WHERE ProdNum = 'PRO/0278';\nExample 7 - Show me all the products created in the year 2023., the SQL command will be something like this SELECT * FROM PRODUCTS WHERE YEAR(ProdDateCreated) = 2023;\nExample 8 - Give me the hsn code of sky tone, the SQL command will be something like this SELECT Prod Name, ProdHSNCode FROM PRODUCTS WHERE ProdName LIKE '%sky tone%' OR ProdDesc LIKE '%sky tone%';\nExample 9 - List the product descriptions for products that have the word 'blue' in their name., the SQL command will be something like this SELECT ProdDesc FROM PRODUCTS WHERE ProdName LIKE '%blue%'; and you can add multiple columns also in sql query for accurate result.\nExample 10 - Tell me which product has highest entries., the SQL command will be something like this SELECT Top 1 ProdName, COUNT(*) AS EntryCount FROM PRODUCTS GROUP BY ProdName ORDER BY EntryCount DESC;\nExample 11 - Tell me which product has second highest entries., the SQL command will be something like this WITH RankedProducts AS (SELECT ProdName, COUNT(*) AS EntryCount,ROW_NUMBER() OVER (ORDER BY COUNT(*) DESC) AS RowNum FROM PRODUCTS GROUP BY ProdName) SELECT ProdName, EntryCount FROM RankedProducts WHERE RowNum = 2;\nalso the sql code should not have ``` in the beginning or end and sql word in output",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [15]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [16]:
config = {"configurable": {"thread_id": "abc345"}}
query = "Hi! I'm Deba."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Hello Deba! How can I assist you today? Do you have any questions about eDominer Technologies, our products, or services?


In [17]:
query = "What is my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Your name is Deba! How can I assist you further?


In [18]:
query = "What is your product?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

We offer Expand smERP, a cloud-based ERP solution specifically designed to streamline operations for mid-sized Indian manufacturers and exporters. 

Here are some key features of Expand smERP:

- Seamless integration with existing business processes.
- Automation of complex tasks to increase efficiency.
- A user-friendly interface that requires minimal training.
- Secure data storage on Microsoft Azure with SSL encryption.
- Integration with popular platforms like WhatsApp, Paytm, and Amazon.
- Customizable options to fit specific business needs.

The benefits of using Expand smERP include improved business efficiency and productivity, reduced costs through automation and streamlined processes, enhanced data security and management, and a scalable solution that can grow with your business.

If you have any specific questions or need more details, feel free to ask!


In [19]:
query = "Can you give its pricing?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Sure! Here are the pricing details for our Expand smERP products:

1. **Expand eziSales: Lead Management**
   - Price: ₹ 0 per month
   - Features include:
     - Create Contact (Unlimited)
     - Capture Leads
     - Create Follow-ups
     - Mobile Notification
     - Call Log (Duration Only)

2. **Expand Lite: Startup Business**
   - Price: ₹ 1800 per concurrent user/month*
   - Features include:
     - Lead Management
     - Sales Planning
     - Order to Cash
     - Procure to Pay
     - Approval Workflow
     - Product Catalogue
     - KPI Dashboard
     - Analytics Dashboard
     - Complete Accounting

3. **Expand smERP: Enterprise Business**
   - Price: ₹ 2500 per concurrent user/month*
   - Includes all features of Expand Lite plus:
     - Jobwork
     - Material Requirement Planning
     - Multi-Level Approval
     - Hand-held Terminal App
     - Customized Reports
     - Vendor Portal
     - Wor

In [20]:
query = "What is the product name for the product with ProdNum PRO/0001?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT ProdName FROM PRODUCTS WHERE ProdNum = 'PRO/0001';


In [21]:
query = "tell me which product has highest entries"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT Top 1 ProdName, COUNT(*) AS EntryCount FROM PRODUCTS GROUP BY ProdName ORDER BY EntryCount DESC;


In [22]:
query = "Can you give its no. of entries?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT COUNT(*) FROM PRODUCTS;


### Managing Conversation History to prevent message Overflow

In [23]:
from langchain_core.messages import SystemMessage, trim_messages

trimmer = trim_messages(
    max_tokens=65,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [24]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke(
        {"messages": trimmed_messages}
    )
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [25]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Tell me the Product Name whose Product back office code is 4COPI047A"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT ProdName FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';


In [26]:
config = {"configurable": {"thread_id": "abc567"}}
query = "tell its product description"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT ProdDesc FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';


In [27]:
config = {"configurable": {"thread_id": "abc567"}}
query = "tell its UOMID"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

SELECT UOMID FROM PRODUCTS WHERE ProdBackOfficeCode = '4COPI047A';


In [28]:
trimmer.invoke(input_messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}, id='51b922df-f696-4d21-8d14-16353ad4392d'),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}, id='910fa06e-33b5-446c-9019-9af690870283'),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}, id='79741e28-dc6b-4100-b94d-44e51b00c300'),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}, id='667a9fe5-dee0-4b7f-ad02-03ad291fcd2e'),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}, id='b846fb30-2722-44b8-9c74-01ced3dbdf82'),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}, id='fe03f111-334e-4613-912e-db3ffe771dda'),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={}, id='be84a5bb-a6cf-4528-9d2d-715d193ec37e'),
 HumanMessage(content='tell its UOMID', additional_kwargs={}, response_metadata={}, id='04c73949-7d57-4ff3-92ff-1f6ff59c0fef')]

In [29]:
config = {"configurable": {"thread_id": "abc567"}}
query = "What SQL query did I ask?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

You asked for the UOMID, but I mistakenly included a condition based on the ProdBackOfficeCode. If you want to retrieve the UOMID for all products, the correct SQL query would be:

SELECT UOMID FROM PRODUCTS;


In [30]:
config = {"configurable": {"thread_id": "abc567"}}
query = "Can you give last response?"

input_messages = messages + [HumanMessage(query)]
output = app.invoke(
    {"messages": input_messages},
    config,
)
output["messages"][-1].pretty_print()

================================== Ai Message ==================================

Sure! Here’s the last response:

You're a good assistant.


### Adding Streaming Features

In [31]:
config = {"configurable": {"thread_id": "abc789"}}
query = "List all products created before 2022-01-01."

input_messages = messages + [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

|SELECT| *| FROM| PRODUCTS| WHERE| Prod|Date|Created| <| '|202|2|-|01|-|01|';||

### Loading Dataset

In [32]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader(r"C:\Users\eDominer\Python Project\ChatBot\ChatBot_with_Database\OpenAI Tuned Model\Help_whole.pdf")

docs = loader.load()
# print(docs[168])

print(docs[0].page_content)


Customer Preference
Customer Preference is used to record the unique ‘preferences’ of the Customer such as unique product identifications, customer
specific clauses, customer specific charges or opening balances of products with the Customer.
How to Create a New Customer Preference?
In order to create a NEW Customer Preference the user must go to
CRM >> Customer Preference.
This opens up the Customer Preference Detail page.
Explanations of the various fields in the Customer Preference Detail page:
Customer:
User must ‘tag’ a Customer in the field labeled Customer.  This is the Customer whose preferences are going to be recorded.
This is a mandatory field and the Customer Preference entry cannot be saved without entering this field.
Payment Terms
The default payment terms for a particular Customer can be tagged in the field labeled Payment Terms.  In order to do this, the
user must click on the small icon beside the field to open up a list of pre-configured Payment Terms.  The user must

In [33]:
print(f"Total characters: {len(docs[0].page_content)}")

Total characters: 3557


### Splitting Documents

In [34]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split blog post into {len(all_splits)} sub-documents.")

Split blog post into 746 sub-documents.


### Loading Data into Vector store

In [35]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

from langchain_core.vectorstores import InMemoryVectorStore
# Initialize with an embedding model
vector_store = InMemoryVectorStore(embeddings)

In [36]:
document_ids = vector_store.add_documents(documents=all_splits)

print(document_ids[:3])

['27cf75d6-f148-4c63-b8bf-30d549b08a92', '36dd6b4f-de13-426c-8e21-022fb1a8cd01', 'f88859f8-22c5-480b-9cce-70ae14c6eb9c']


### Retrieval and Generation

In [37]:
from langchain import hub
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

In [38]:
# Define prompt for question-answering
prompt = hub.pull("rlm/rag-prompt")

# Define state for application
class State(TypedDict):
    question: str
    context: List[Document]
    answer: str


# Define application steps
def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = model.invoke(messages)
    return {"answer": response.content}

c:\Users\eDominer\anaconda3\envs\py310\lib\site-packages\langsmith\client.py:261: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [39]:
# Compile application and test
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()

In [40]:
response = graph.invoke({"question": "What is Customer Preference?"})
print(response["answer"])

Customer Preference refers to the specific preferences recorded for a customer, including unique product identifications, customer-specific clauses, charges, and opening balances. This information is essential for tailoring services and products to meet individual customer needs. To create a Customer Preference, a user must access the Customer Preference Detail page in the CRM system and fill in the necessary fields.


In [41]:
response = graph.invoke({"question": "How to Create a New Customer Preference?"})
print(response["answer"])

To create a new Customer Preference, go to CRM and click on Customer Preference, which will open the Customer Preference Detail page. Tag the required customer in the mandatory Customer field and fill in any other necessary details. Finally, verify all data and click on ‘Save’ to complete the process.


In [42]:
response = graph.invoke({"question": "How to Create Machine master?"})
print(response["answer"])

To create a Machine master, go to the Business Definition module and click on the Machine option. Click on the new button to enter details such as Machine No., Machine Name, Description, and costs, and ensure to activate the machine if needed. Additionally, you can add installation dates and other details in the Advance tab.


In [43]:
response = graph.invoke({"question": "How to create contact?"})
print(response["answer"])

To create a contact, go to CRM, click on Contact, then click on New. Enter the contact name, add phone numbers, addresses, and emails in their respective tabs, and click Save once all relevant data is entered. You can also edit or delete contacts later as needed.


In [44]:
response = graph.invoke({"question": "How to create Export Invoice?"})
print(response["answer"])

To create an Export Invoice, go to the Export module and click on Export Invoice/Packing List, then select New. After tagging the relevant PI/LC or sales order in the document tab, fill in the necessary details in the Basic and Advance tabs before saving the invoice. Optionally, you can generate an Export Invoice with GST by selecting the Export Invoice GST menu and following the specified steps.


In [45]:
response = graph.invoke({"question": "How to create  PI/LC document?"})
print(response["answer"])

To create a PI/LC document, go to the Export Module and click on the PI/LC menu, then select the New option. Fill in the required details, including customer information, reference numbers, payment terms, and any letter of credit details if applicable. Ensure to save your entries, as they will auto-populate in the Export Invoice when tagged.
